# Prompting a Large Language Model


This is an example notebook for running an open source model from Hugging Face.

In [ ]:
%pip install -r requirements.txt

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

We use the [Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct) in this notebook due to its small size.

In [3]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="cuda", # change to cpu is running locally
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [5]:
def generate(messages):
  pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
  )

  generation_args = {
      "max_new_tokens": 500,
      "return_full_text": False,
      "temperature": 0.0,
      "do_sample": False,
  }
  output = pipe(messages, **generation_args)
  print(output[0]['generated_text'])

In [10]:
template = {
    "role": "a professional Travel Consultant",
    "goal": "You help clients design unique travel itineraries, considering their preferences, budget, and interests",
    "tone": "Your response should be concise and accurate."
}
user_message = "I am traveling with my husband and my child to Helsinki for four days. Plan our trip."

# Basic Prompt

In [12]:
system_prompt = "You are a helpful and concise assistant."
messages = [{
    "role": "system",
    "content": system_prompt},
     {
    "role": "user",
    "content": user_message
}]

generate(messages)

Device set to use cuda
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Here's a suggested itinerary for a four-day trip to Helsinki with your husband and child, focusing on family-friendly activities, cultural experiences, and relaxation:

**Day 1: Arrival and Exploring the City**

- Morning:
  - Arrive at Helsinki Airport and check into your hotel.
  - Take a leisurely stroll around the neighborhood to get acclimated to the city.
  - Have lunch at a local café, such as "Keski-Pakola" for traditional Finnish cuisine.

- Afternoon:
  - Visit the Suuripuisto (Big Park) for a relaxing walk and a chance to see the city's iconic red-brick buildings.
  - Head to the nearby Puistopuisto (Southern Park) for a picnic with Finnish snacks like marinated herring, rye bread, and cured salmon.

- Evening:
  - Explore the Esplanadi (Esplanade) area, which is lined with cafes, restaurants, and shops.
  - Enjoy dinner at "Keskeinen" for a family-friendly atmosphere and Finnish dishes.

**Day 2: Museums and Culture**

- Morning:
  - Visit the Finnish National Museum (Kans

#Zero-shot Prompting

In [11]:
system_prompt = f"You are {template['role']}. {template['goal']}. {template['tone']}"
messages = [{
    "role": "system",
    "content": system_prompt},
     {
    "role": "user",
    "content": user_message
}]

generate(messages)

Device set to use cuda
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Day 1: Arrival and Exploring the City
- Arrive at Helsinki Airport and check into your hotel.
- Take a leisurely stroll around the city center, visiting the Senate Square and the National Museum of Finland.
- Enjoy a family dinner at a local restaurant, such as "Keski-Suomessa" for traditional Finnish cuisine.

Day 2: Suomenlinna and Harbor Tour
- Visit Suomenlinna, a UNESCO World Heritage site, and explore the historic fortress and its museums.
- Take a harbor cruise to see the city's beautiful architecture and waterfront.
- Have lunch at "Kupittaa" on Market Square, offering a variety of Finnish dishes.
- In the afternoon, visit the Sibelius Museum and enjoy a relaxing walk in the nearby park.

Day 3: Helsinki Zoo and Temppeliaukio Church
- Spend the morning at Helsinki Zoo, where your child can enjoy seeing a variety of animals.
- After lunch at "Kesäkankaan" for a Finnish-style lunch, head to Temppeliaukio Church, known for its unique acoustics and stunning architecture.
- Visit t

# One-shot Prompt



In [13]:
system_prompt = f"You are {template['role']}. {template['goal']}. {template['tone']}"
messages = [{
    "role": "system",
    "content": system_prompt},
    {
    "role": "user",
    "content": "Customer: We are traveling to Paris. We are interested in art and fine dining."
    },
    {
    "role": "assistant",
    "content": "Travel Consultant: - Accommodation: Hôtel Le Meurice ($850/night).\n- Activities: Louvre ($17/person) and Musée d'Orsay ($16/person).\n- Dining: Le Jules Verne ($300/person).\n- Transport: Paris Visite Pass ($26/person for 2 days); metro travel under 30 mins.\n- Total Cost: ~$2,000 for two (excluding airfare)."
    },
    {
    "role": "user",
    "content": "Customer:" + user_message
}]

generate(messages)

Device set to use cuda
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Travel Consultant:

Day 1:
- Arrival at Helsinki Airport.
- Check-in at Hotel Rekka ($150/night for a family suite).
- Dinner at Kodisma ($50/person).

Day 2:
- Visit Suur-Ilmailualue (Sun Archipelago) ($20/person for boat ticket).
- Lunch at The Coffee House ($30/person).
- Afternoon visit to the Finnish National Museum ($10/person).
- Dinner at The Coffee House.

Day 3:
- Morning visit to the Suomenlinna Sea Fortress ($10/person).
- Lunch at The Coffee House.
- Afternoon visit to the Temppeliaukio Church (Rock Church) ($10/person).
- Dinner at The Coffee House.

Day 4:
- Morning visit to the Finnish Museum of Photography ($10/person).
- Lunch at The Coffee House.
- Afternoon visit to the Sibelius Monument ($10/person).
- Departure from Helsinki Airport.

Transport: Public transportation is efficient and affordable in Helsinki. A 4-day Helsinki Card ($50/person) provides unlimited travel on buses, trams, and ferries.

Total Cost (excluding airfare): ~$1,500 for accommodation, meals, 

# Two-shots Prompt

In [14]:
system_prompt = f"You are {template['role']}. {template['goal']}. {template['tone']}"
messages = [{
    "role": "system",
    "content": system_prompt},
    {
    "role": "user",
    "content": "Customer: We are traveling to Paris. We are interested in art and fine dining."
    },
    {
    "role": "assistant",
    "content": "Travel Consultant: - Accommodation: Hôtel Le Meurice ($850/night).\n- Activities: Louvre ($17/person) and Musée d'Orsay ($16/person).\n- Dining: Le Jules Verne ($300/person).\n- Transport: Paris Visite Pass ($26/person for 2 days); metro travel under 30 mins.\n- Total Cost: ~$2,000 for two (excluding airfare)."
    },
    {
    "role": "user",
    "content": "Customer: I am traveling with my best friend to Cairo for a weekend. We are interested in history and local cuisine on a budget."
    },
    {
    "role": "assistant",
    "content": "Travel Consultant: - Accommodation: Steigenberger El Tahrir Hotel ($70/night).\n- Activities: Giza Pyramids and Sphinx tour ($20/person) and Egyptian Museum ($5/person).\n- Dining: Abou Tarek for koshary ($10/person) and Felfela for traditional dishes ($15/person).\n- Transport: Uber rides and public buses (approx. $10/day).\n- Total Cost: ~$300 for two (excluding airfare)."
    },
    {
    "role": "user",
    "content": "Customer:" + user_message
}]

generate(messages)

Device set to use cuda


 Travel Consultant:

Day 1:
- Arrival and check-in at the Design Hotel (~$150/night).
- Visit Suuripuisto Park for a relaxing afternoon.
- Dinner at the family-friendly restaurant, The Coffee House (~$30/person).

Day 2:
- Visit the Suomenlinna Sea Fortress (~$10/person).
- Lunch at the nearby restaurant, Kupittaa (~$20/person).
- Visit the Finnish National Museum (~$10/person).
- Dinner at the family-friendly restaurant, The Coffee House (~$30/person).

Day 3:
- Visit the Temppeliaukio Church (~$10/person).
- Lunch at the nearby restaurant, Kupittaa (~$20/person).
- Visit the Market Square and enjoy shopping.
- Dinner at the family-friendly restaurant, The Coffee House (~$30/person).

Day 4:
- Visit the Ateneum Art Museum (~$10/person).
- Lunch at the nearby restaurant, Kupittaa (~$20/person).
- Visit the Puijo Observation Tower (~$10/person).
- Farewell dinner at the family-friendly restaurant, The Coffee House (~$30/person).

Transport: Uber rides and public buses (approx. $50/day).

# Chain of thoughts Prompting

In [15]:
system_prompt = f"You are {template['role']}. {template['goal']}. {template['tone']}. " + "Think step by step before giving any suggestions. Consider any factors that may affect the experience at the destination. Think of many plans then choose the best ones."
messages = [{
    "role": "system",
    "content": system_prompt},
     {
    "role": "user",
    "content": user_message
}]

generate(messages)


Device set to use cuda
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Day 1: Arrival and Exploring the City
- Arrive at Helsinki Airport and check into your hotel.
- Take a leisurely stroll around the city center, visiting the Senate Square and the National Museum of Finland.
- Enjoy a family dinner at a local restaurant, such as "Keski-Suomessa" for traditional Finnish cuisine.

Day 2: Helsinki's Waterfront and Museums
- Start the day with a visit to Suomenlinna, a UNESCO World Heritage site, and explore the historic fortress.
- Have lunch at a nearby restaurant, such as "Kaapelitehdas."
- In the afternoon, visit the Finnish National Gallery and the Ateneum Art Museum.
- Enjoy a relaxing evening walk along the Esplanadi Park and dine at "Kupittaa."

Day 3: Day Trip to the Archipelago
- Take a morning ferry to the Helsinki Archipelago, a UNESCO Biosphere Reserve.
- Explore the islands, such as Suomenlinna, Seurasaaren, and Linnansaari.
- Enjoy a picnic lunch on one of the islands.
- Return to Helsinki in the afternoon and have dinner at "Kaapelitehdas."